In [1]:
import pandas as pd
import numpy as np
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.cluster import KMeans
from sklearn.metrics import silhouette_score
import matplotlib.pyplot as plt
from pathlib import Path
import textwrap
import cga_utils
import math
from typing import Iterable, List, Optional, Tuple
import hdbscan
from sklearn.feature_extraction import DictVectorizer
import cluster_features
%load_ext autoreload
%autoreload 2

# Used in the paper

In [4]:
len(pd.read_csv('res/gemma3n_v18.csv'))

497

In [5]:
import numpy as np
from sklearn.preprocessing import StandardScaler
import math

EPS = 1e-9

def value_bucket(ans, pred):
    if ans == pred:
        return 0
    if ans == -1*pred:
        return 1
    if math.isnan(pred):
        return 2
    else: 
        return 3


errors = pd.read_csv('res/gemma3n_v18.csv').query('exact_match == False')
print("Error set n:", len(errors), ", Q1: ", len(errors)/4)
# extra features
errors["ratio"] = errors.apply(lambda row: (row["pred"] / (row["answer"]+EPS)) if row["answer"] != 0 else np.inf,  axis=1 )
errors["x100_flag"] = errors["ratio"].apply(lambda ratio: int(0.95 < ratio/100 < 1.05 or 0.95 < ratio*100 < 1.05))

errors["scale"] = errors["scale"].fillna("")
errors["pred_scale"] = errors["pred_scale"].fillna("")
errors["code_calc_pattern"]= errors["code_calc_pattern"].fillna("nan")

errors["scale_mismatch"]= errors["scale"] != errors["pred_scale"]
errors["value_mismatch"]= errors["answer"] != errors["pred"]
errors["value_nan"]= errors["pred"].isna()
errors["value_match"]= errors.apply(lambda row: value_bucket(row["answer"], row["pred"]) ,  axis=1 )

errors["calc_pattern"] = cga_utils.aggregate_calc_patterns(errors["calc_pattern"],3) 
errors["code_calc_pattern"] = cga_utils.aggregate_calc_patterns(errors["code_calc_pattern"],3) 
#convert
#vects = errors[["calc_pattern", "code_calc_pattern", "scale", "pred_scale",'sign_error', "x100_flag"]]
vects = errors[["calc_pattern", "code_calc_pattern", "scale_mismatch", "value_match", "error_code"]]

X_dict = vects.to_dict(orient="records")
dv = DictVectorizer(sparse=False)

X_features = dv.fit_transform(X_dict).astype(int)

setups = cluster_features.try_hdbscan(X_features)
print(setups)
#print([ (a,(b,c,d,e)) for (a,b) in setups.iterrows()])
clusterer = hdbscan.HDBSCAN(min_cluster_size=2, min_samples=1, metric="hamming")
clusterer.fit_predict(X_features)

errors["cluster"] = clusterer.labels_

counts = errors["cluster"].value_counts()
print(counts)
errors.to_csv('res/g3n_errors_18_cv1_2_1.csv')
for id in counts.index[:]:    
#errors[["question","calc_pattern","code_calc_pattern","derivation", "answer","pred" ,'scale', 'pred_scale', 'sign_error', "x100_flag", "error_code",  "cluster"]].query('cluster == cluster[cluster != -1].value_counts().idxmax()')    
    print(id)
    print(errors[["question","calc_pattern","code_calc_pattern","derivation", "answer","pred" ,'scale', 'pred_scale', 'sign_error', "x100_flag", "error_code",  "cluster"]].query(f'cluster == {id}'))
len (errors)

Error set n: 290 , Q1:  72.5
merging
merging
    min_cluster_size  min_samples  n_clusters  noise_ratio
0                  2            1          64     0.134483
10                 4            1          29     0.165517
5                  3            1          36     0.179310
1                  2            2          39     0.203448
15                 5            1          24     0.206897
6                  3            2          35     0.213793
11                 4            2          26     0.213793
2                  2            3          43     0.213793
20                 6            1          21     0.234483
16                 5            2          19     0.255172
25                10            1          11     0.255172
21                 6            2          15     0.282759
7                  3            3          27     0.289655
4                  2            5          38     0.306897
3                  2            4          34     0.313793
12         

290

In [6]:
#errors[["question","calc_pattern","code_calc_pattern",'scale', 'pred_scale', 'sign_error', "x100_flag", "error_code",  "cluster"]].query(f'cluster == 37').to_csv('res/gemma3_alg_step1.csv')
errors.query(f'cluster == 37').to_csv('res/gemma3n_alg_step1.csv')

In [9]:
import numpy as np
from sklearn.preprocessing import StandardScaler
import math

EPS = 1e-9

def value_bucket(ans, pred):
    if ans == pred:
        return 0
    if ans == -1*pred:
        return 1
    if math.isnan(pred):
        return 2
    else: 
        return 3


errors = pd.read_csv('res/gemma3n_v19a_r1.csv').query('exact_match == False')
print("Error set n:", len(errors), ", Q1: ", len(errors)/4)
# extra features
errors["ratio"] = errors.apply(lambda row: (row["pred"] / (row["answer"]+EPS)) if row["answer"] != 0 else np.inf,  axis=1 )
errors["x100_flag"] = errors["ratio"].apply(lambda ratio: int(0.95 < ratio/100 < 1.05 or 0.95 < ratio*100 < 1.05))

errors["scale"] = errors["scale"].fillna("")
errors["pred_scale"] = errors["pred_scale"].fillna("")
errors["code_calc_pattern"]= errors["code_calc_pattern"].fillna("nan")

errors["scale_mismatch"]= errors["scale"] != errors["pred_scale"]
errors["value_mismatch"]= errors["answer"] != errors["pred"]
errors["value_nan"]= errors["pred"].isna()
errors["value_match"]= errors.apply(lambda row: value_bucket(row["answer"], row["pred"]) ,  axis=1 )

errors["calc_pattern"] = cga_utils.aggregate_calc_patterns(errors["calc_pattern"],3) 
errors["code_calc_pattern"] = cga_utils.aggregate_calc_patterns(errors["code_calc_pattern"],3) 
#convert
#vects = errors[["calc_pattern", "code_calc_pattern", "scale", "pred_scale",'sign_error', "x100_flag"]]
vects = errors[["calc_pattern", "code_calc_pattern", "scale_mismatch", "value_match", "error_code"]]

X_dict = vects.to_dict(orient="records")
dv = DictVectorizer(sparse=False)

X_features = dv.fit_transform(X_dict).astype(int)

setups = cluster_features.try_hdbscan(X_features)
print(setups)
#print([ (a,(b,c,d,e)) for (a,b) in setups.iterrows()])
clusterer = hdbscan.HDBSCAN(min_cluster_size=2, min_samples=1, metric="hamming")
clusterer.fit_predict(X_features)

errors["cluster"] = clusterer.labels_

counts = errors["cluster"].value_counts()
print(counts)
errors.to_csv('res/g3n_errors_19a_cv1_2_1.csv')
for id in counts.index[:]:    
#errors[["question","calc_pattern","code_calc_pattern","derivation", "answer","pred" ,'scale', 'pred_scale', 'sign_error', "x100_flag", "error_code",  "cluster"]].query('cluster == cluster[cluster != -1].value_counts().idxmax()')    
    print(id)
    print(errors[["question","calc_pattern","code_calc_pattern","derivation", "answer","pred" ,'scale', 'pred_scale', 'sign_error', "x100_flag", "error_code",  "cluster"]].query(f'cluster == {id}'))
len (errors)

Error set n: 269 , Q1:  67.25
merging
merging
    min_cluster_size  min_samples  n_clusters  noise_ratio
0                  2            1          58     0.159851
27                10            3           8     0.163569
32                15            3           6     0.163569
31                15            2           5     0.189591
1                  2            2          46     0.193309
5                  3            1          38     0.197026
30                15            1           5     0.197026
20                 6            1          17     0.204461
6                  3            2          37     0.211896
2                  2            3          40     0.215613
15                 5            1          22     0.219331
26                10            2           7     0.226766
10                 4            1          27     0.230483
7                  3            3          30     0.237918
11                 4            2          28     0.241636
28        

269

In [8]:
#errors[["question","calc_pattern","code_calc_pattern",'scale', 'pred_scale', 'sign_error', "x100_flag", "error_code",  "cluster"]].query(f'cluster == 37').to_csv('res/gemma3_alg_step1.csv')
errors.query(f'cluster == 50').to_csv('res/gemma3n_alg_step2.csv')

In [11]:
#errors[["question","calc_pattern","code_calc_pattern",'scale', 'pred_scale', 'sign_error', "x100_flag", "error_code",  "cluster"]].query(f'cluster == 37').to_csv('res/gemma3_alg_step1.csv')
errors.query(f'cluster == 43').to_csv('res/gemma3n_alg_step2b.csv')

In [12]:
import numpy as np
from sklearn.preprocessing import StandardScaler
import math

EPS = 1e-9

def value_bucket(ans, pred):
    if ans == pred:
        return 0
    if ans == -1*pred:
        return 1
    if math.isnan(pred):
        return 2
    else: 
        return 3


errors = pd.read_csv('res/gemma3n_v21_r1.csv').query('exact_match == False')
print("Error set n:", len(errors), ", Q1: ", len(errors)/4)
# extra features
errors["ratio"] = errors.apply(lambda row: (row["pred"] / (row["answer"]+EPS)) if row["answer"] != 0 else np.inf,  axis=1 )
errors["x100_flag"] = errors["ratio"].apply(lambda ratio: int(0.95 < ratio/100 < 1.05 or 0.95 < ratio*100 < 1.05))

errors["scale"] = errors["scale"].fillna("")
errors["pred_scale"] = errors["pred_scale"].fillna("")
errors["code_calc_pattern"]= errors["code_calc_pattern"].fillna("nan")

errors["scale_mismatch"]= errors["scale"] != errors["pred_scale"]
errors["value_mismatch"]= errors["answer"] != errors["pred"]
errors["value_nan"]= errors["pred"].isna()
errors["value_match"]= errors.apply(lambda row: value_bucket(row["answer"], row["pred"]) ,  axis=1 )

errors["calc_pattern"] = cga_utils.aggregate_calc_patterns(errors["calc_pattern"],3) 
errors["code_calc_pattern"] = cga_utils.aggregate_calc_patterns(errors["code_calc_pattern"],3) 
#convert
#vects = errors[["calc_pattern", "code_calc_pattern", "scale", "pred_scale",'sign_error', "x100_flag"]]
vects = errors[["calc_pattern", "code_calc_pattern", "scale_mismatch", "value_match", "error_code"]]

X_dict = vects.to_dict(orient="records")
dv = DictVectorizer(sparse=False)

X_features = dv.fit_transform(X_dict).astype(int)

setups = cluster_features.try_hdbscan(X_features)
print(setups)
#print([ (a,(b,c,d,e)) for (a,b) in setups.iterrows()])
clusterer = hdbscan.HDBSCAN(min_cluster_size=2, min_samples=1, metric="hamming")
clusterer.fit_predict(X_features)

errors["cluster"] = clusterer.labels_

counts = errors["cluster"].value_counts()
print(counts)
errors.to_csv('res/g3n_errors_21_cv1_2_1.csv')
for id in counts.index[:]:    
#errors[["question","calc_pattern","code_calc_pattern","derivation", "answer","pred" ,'scale', 'pred_scale', 'sign_error', "x100_flag", "error_code",  "cluster"]].query('cluster == cluster[cluster != -1].value_counts().idxmax()')    
    print(id)
    print(errors[["question","calc_pattern","code_calc_pattern","derivation", "answer","pred" ,'scale', 'pred_scale', 'sign_error', "x100_flag", "error_code",  "cluster"]].query(f'cluster == {id}'))
len (errors)

Error set n: 248 , Q1:  62.0
merging
merging
    min_cluster_size  min_samples  n_clusters  noise_ratio
31                15            2           6     0.048387
20                 6            1          16     0.096774
25                10            1          11     0.120968
0                  2            1          56     0.129032
5                  3            1          32     0.157258
21                 6            2          15     0.161290
10                 4            1          26     0.161290
26                10            2          11     0.165323
6                  3            2          31     0.193548
1                  2            2          39     0.201613
30                15            1           7     0.205645
11                 4            2          25     0.213710
15                 5            1          21     0.221774
4                  2            5          28     0.233871
22                 6            3          13     0.245968
16         

248

In [15]:
#errors[["question","calc_pattern","code_calc_pattern",'scale', 'pred_scale', 'sign_error', "x100_flag", "error_code",  "cluster"]].query(f'cluster == 37').to_csv('res/gemma3_alg_step1.csv')
errors.query(f'cluster == 22').to_csv('res/gemma3n_alg_step3.csv')

In [16]:
#errors[["question","calc_pattern","code_calc_pattern",'scale', 'pred_scale', 'sign_error', "x100_flag", "error_code",  "cluster"]].query(f'cluster == 37').to_csv('res/gemma3_alg_step1.csv')
errors.query(f'cluster == 54').to_csv('res/gemma3n_alg_step3b.csv')

In [17]:
#errors[["question","calc_pattern","code_calc_pattern",'scale', 'pred_scale', 'sign_error', "x100_flag", "error_code",  "cluster"]].query(f'cluster == 37').to_csv('res/gemma3_alg_step1.csv')
errors.query(f'cluster == 23').to_csv('res/gemma3n_alg_step3c.csv')